In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

import xgboost as xgb
from xgboost.sklearn import XGBClassifier

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import IsolationForest

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import classification_report

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/speed-dating/Speed Dating Data.csv


# Exploratory Data Analysis

In [2]:
df = pd.read_csv("../input/speed-dating/Speed Dating Data.csv",encoding='ISO-8859-1')
df = df.drop(["id", "iid"], axis = 1)

In [3]:
df.shape

(8378, 193)

In [4]:
df.head()

,gender,idg,condtn,wave,round,position,positin1,order,partner,pid,...,attr3_3,sinc3_3,intel3_3,fun3_3,amb3_3,attr5_3,sinc5_3,intel5_3,fun5_3,amb5_3
0,0,1,1,1,10,7,NaN,4,1,11.0,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
1,0,1,1,1,10,7,NaN,3,2,12.0,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
2,0,1,1,1,10,7,NaN,10,3,13.0,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
3,0,1,1,1,10,7,NaN,5,4,14.0,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
4,0,1,1,1,10,7,NaN,7,5,15.0,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN


In [5]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
gender,8378.0,0.500597,0.500029,0.0,0.0,1.0,1.0,1.0
idg,8378.0,17.327166,10.940735,1.0,8.0,16.0,26.0,44.0
condtn,8378.0,1.828837,0.376673,1.0,2.0,2.0,2.0,2.0
wave,8378.0,11.350919,5.995903,1.0,7.0,11.0,15.0,21.0
round,8378.0,16.872046,4.358458,5.0,14.0,18.0,20.0,22.0
...,...,...,...,...,...,...,...,...
attr5_3,2016.0,6.810020,1.507341,2.0,6.0,7.0,8.0,10.0
sinc5_3,2016.0,7.615079,1.504551,2.0,7.0,8.0,9.0,10.0
intel5_3,2016.0,7.932540,1.340868,4.0,7.0,8.0,9.0,10.0
fun5_3,2016.0,7.155258,1.672787,1.0,6.0,7.0,8.0,10.0


In [6]:
# Attributes
df.columns.values

array(['gender', 'idg', 'condtn', 'wave', 'round', 'position', 'positin1',
       'order', 'partner', 'pid', 'match', 'int_corr', 'samerace',
       'age_o', 'race_o', 'pf_o_att', 'pf_o_sin', 'pf_o_int', 'pf_o_fun',
       'pf_o_amb', 'pf_o_sha', 'dec_o', 'attr_o', 'sinc_o', 'intel_o',
       'fun_o', 'amb_o', 'shar_o', 'like_o', 'prob_o', 'met_o', 'age',
       'field', 'field_cd', 'undergra', 'mn_sat', 'tuition', 'race',
       'imprace', 'imprelig', 'from', 'zipcode', 'income', 'goal', 'date',
       'go_out', 'career', 'career_c', 'sports', 'tvsports', 'exercise',
       'dining', 'museums', 'art', 'hiking', 'gaming', 'clubbing',
       'reading', 'tv', 'theater', 'movies', 'concerts', 'music',
       'shopping', 'yoga', 'exphappy', 'expnum', 'attr1_1', 'sinc1_1',
       'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1', 'attr4_1', 'sinc4_1',
       'intel4_1', 'fun4_1', 'amb4_1', 'shar4_1', 'attr2_1', 'sinc2_1',
       'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1', 'attr3_1', 'sinc3_1',
     

- gender: Gender of self
- age: Age of self
- age_o: Age of partner
- d_age: Difference in age
- race: Race of self
- race_o: Race of partner
- samerace: Whether the two persons have the same race or not.
- importancesamerace: How important is it that partner is of same race?
- importancesamereligion: How important is it that partner has same religion?
- field: Field of study
- prefoattractive: How important does partner rate attractiveness
- prefosinsere: How important does partner rate sincerity
- prefointelligence: How important does partner rate intelligence
- prefofunny: How important does partner rate being funny
- prefoambitious: How important does partner rate ambition
- prefoshared_interests: How important does partner rate having shared interests
- attractive_o: Rating by partner (about me) at night of event on attractiveness
- sincere_o: Rating by partner (about me) at night of event on sincerity
- intelligence_o: Rating by partner (about me) at night of event on intelligence
- funny_o: Rating by partner (about me) at night of event on being funny
- ambitous_o: Rating by partner (about me) at night of event on being ambitious
- sharedinterestso: Rating by partner (about me) at night of event on shared interest
- attractive_important: What do you look for in a partner - attractiveness
- sincere_important: What do you look for in a partner - sincerity
- intellicence_important: What do you look for in a partner - intelligence
- funny_important: What do you look for in a partner - being funny
- ambtition_important: What do you look for in a partner - ambition
- sharedinterestsimportant: What do you look for in a partner - shared interests
- attractive: Rate yourself - attractiveness
- sincere: Rate yourself - sincerity
- intelligence: Rate yourself - intelligence
- funny: Rate yourself - being funny
- ambition: Rate yourself - ambition
- attractive_partner: Rate your partner - attractiveness
- sincere_partner: Rate your partner - sincerity
- intelligence_partner: Rate your partner - intelligence
- funny_partner: Rate your partner - being funny
- ambition_partner: Rate your partner - ambition
- sharedinterestspartner: Rate your partner - shared interests
- sports: Your own interests [1-10]
- tvsports
- exercise
- dining
- museums
- art
- hiking
- gaming
- clubbing
- reading
- tv
- theater
- movies
- concerts
- music
- shopping
- yoga
- interests_correlate: Correlation between participant’s and partner’s ratings of interests.
- expectedhappywithsdpeople: How happy do you expect to be with the people you meet during the speed-dating event?
- expectednuminterestedinme: Out of the 20 people you will meet, how many do you expect will be interested in dating you?
- expectednummatches: How many matches do you expect to get?
- like: Did you like your partner?
- guessprobliked: How likely do you think it is that your partner likes you?
- met: Have you met your partner before?
- decision: Decision at night of event.
- decision_o: Decision of partner at night of event.
- match: Match (yes/no)

In [7]:
# Percentage of matches (0: no match, 1: match)
print(100*df['match'].value_counts() / df.shape[0])

0    83.528288
1    16.471712
Name: match, dtype: float64


# Preprocess Data

In [8]:
df['income'] = df['income'].str.replace(",","").astype(float)

In [9]:
df_corr = df.corr().dropna()

In [10]:
subset_index = df_corr["match"].abs().sort_values(ascending = False)[:11].index.tolist() + ["income"]
df_subset = df[subset_index]
df_subset_corr = df_subset.corr()

In [11]:
# Fix missing value
df_subset.isna().sum()

match        0
dec_o        0
dec          0
like_o     250
like       240
fun_o      360
fun        350
shar_o    1076
shar      1067
attr_o     212
attr       202
income    4099
dtype: int64

In [12]:
null_variable = [
    'like_o', 'like', 'fun_o',
    'fun', 'shar_o', 'shar',
    'attr_o', 'attr', 'income'
]

In [13]:
impute_it = IterativeImputer()
impute_data = impute_it.fit_transform(df_subset[null_variable])
df_subset[null_variable] = impute_data

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


In [14]:
print(df_subset.isna().sum().sum())

0


In [15]:
# Outlier detection


ifd = IsolationForest(n_estimators=50, warm_start=True)
ifd.fit(df_subset)  

print(df_subset.shape)

(8378, 12)


In [16]:
df_subset = df_subset[ifd.predict(df_subset) == 1]

print(df_subset.shape)

(5907, 12)


In [17]:
# Normalize Data
X = df_subset.drop(['match'], axis = 1)
y = df_subset['match'].reset_index(drop = True)

print(X.shape, y.shape)

(5907, 11) (5907,)


# Split Data into Training and Testing Sets

In [18]:
min_max_scaler = MinMaxScaler()
X = min_max_scaler.fit_transform(X,y)

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 16)

In [20]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(4725, 11) (1182, 11) (4725,) (1182,)


In [21]:
xgbc = xgb.XGBClassifier()
xgbc.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [22]:
score = xgbc.score(X_train, y_train)
print("Training score: ", score)

Training score:  1.0


In [23]:
cv_score = cross_val_score(xgbc, X_train, y_train, cv=5)
print("CV mean score: ", cv_score.mean())

CV mean score:  1.0


In [24]:
y_pred = xgbc.predict(X_test)
y_predictions = [float(value) for value in y_pred]

In [25]:
cr = classification_report(y_test,y_predictions, digits = 4)
print(cr)

              precision    recall  f1-score   support

           0     1.0000    1.0000    1.0000      1069
           1     1.0000    1.0000    1.0000       113

    accuracy                         1.0000      1182
   macro avg     1.0000    1.0000    1.0000      1182
weighted avg     1.0000    1.0000    1.0000      1182



In [26]:
y_pred = xgbc.predict(X_test)
y_predictions = [float(value) for value in y_pred]

In [27]:
cr = classification_report(y_test,y_predictions, digits = 4)
print(cr)

              precision    recall  f1-score   support

           0     1.0000    1.0000    1.0000      1069
           1     1.0000    1.0000    1.0000       113

    accuracy                         1.0000      1182
   macro avg     1.0000    1.0000    1.0000      1182
weighted avg     1.0000    1.0000    1.0000      1182

